In [1]:
pip install imutils 

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install scipy

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import imutils
from imutils.perspective import four_point_transform
blue = (255,0,0)
green = (0,255,0)

In [5]:
# Loading and Displaying Image
path = "/Document-Scanner"
img = cv.imread("/Document-Scanner/1.jpg")
img = cv.resize(img, (600,800))
orig_image = img.copy()

In [6]:
# Blurring and Edge Detection
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)# convert the image to gray scale
blur = cv.GaussianBlur(gray, (5, 5), 0) # Add Gaussian blur
cv.imwrite(path + "result1blur.jpg", blur)
edged = cv.Canny(blur, 75, 200) # Apply the Canny algorithm to find the edges
cv.imwrite(path + "result2canny.jpg", edged)



True

In [7]:
# Getting Contours
(contours, _) = cv.findContours(edged.copy(), cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv.contourArea, reverse=True)
cv.drawContours(img, contours, -1, blue, 3)
cv.imwrite(path + "result3drawcountour.jpg", img)  

True

In [8]:
# Selecting Highest Contour
# go through each contour
for c in contours:
    peri = cv.arcLength(c, True)
    approx = cv.approxPolyDP(c, 0.015 * peri, True)
    if len(approx) == 4:
        count = approx
        break
     # if found a countour with 4 points break the for loop
    # Assuming document is found
    # Drawing the contours on the original image
cv.drawContours(img, [count], -1, green, 3)
cv.imwrite(path + "result4selectedcountour.jpg", img)  


True

In [9]:
# Finding Right Points
# Create 4 lists with 2 places in the list represting x,y
points = count.reshape(4, 2) 
input_points = np.zeros((4, 2), dtype="float32") # Empty array to store corner points, same shape

points_sum = points.sum(axis=1) # summing x and y
input_points[0] = points[np.argmin(points_sum)] # Top left
input_points[3] = points[np.argmax(points_sum)] # Bottom Right

points_diff = np.diff(points, axis=1)  # Difference x and y
input_points[1] = points[np.argmin(points_diff)] # top right
input_points[2] = points[np.argmax(points_diff)] # bottom left

# Dimension of new image = distance between point coordinates
(tl, tr, br, bl) = input_points
bottom_width = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
top_width = np.sqrt(((tr[0] - tr[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
right_height = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
left_height = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))



In [10]:
# Calculate output image size
# Use max width and height
max_width = max(int(bottom_width), int(top_width))
#max_height = max(int(right_height), int(left_height))
max_height = int(max_width * 1.414) # Aspect ratio for A4 1 :1.4142
# Desired Point Values
converted_points = np.float32([[0, 0], [max_width, 0], [0, max_height], [max_width, max_height]])
print ("Matrix",converted_points)

Matrix [[0. 0.]
 [2. 0.]
 [0. 2.]
 [2. 2.]]


In [11]:
# Perspective Transformation
matrix = cv.getPerspectiveTransform(input_points, converted_points)
warped = cv.warpPerspective(orig_image, matrix, (max_width, max_height))
cv.imwrite(path + "result5Final.jpg", warped)

True

In [12]:
# Binary Image
imgs = cv.cvtColor(warped, cv.COLOR_BGR2GRAY)
th3 = cv.adaptiveThreshold(imgs, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2);
cv.imwrite(path + "result6Tresh.jpg", th3)

True